<a href="https://colab.research.google.com/github/nevencaplar/JaxPeriodDrwFit/blob/main/JaxAstronomy_save_compile_and_pad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax
import jax.numpy as jnp



import logging


from jax._src import compilation_cache as cc

print("cpu/gpu: " + str(jax.default_backend()))
cc.set_cache_dir("./cache_min_example")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)
jax.config.update("jax_explain_cache_misses", True)


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


cpu/gpu: cpu


In [ ]:
# test function, just
@jax.jit
def test_fun(z):
  def fn(y):
    y -= jax.nn.logsumexp(y,0)
    y -= jax.nn.logsumexp(y,1)
    return y
  for _ in range(20): z = fn(z)
  return z

In [ ]:
# https://jax.readthedocs.io/en/latest/_autosummary/jax.random.gumbel.html
a = jax.random.gumbel(jax.random.PRNGKey(0),(100,100))

DEBUG:jax._src.dispatch:Finished tracing + transforming convert_element_type for pjit in 0.001432657 sec
  never seen function:
    convert_element_type id=133816218245088 defined at /usr/local/lib/python3.10/dist-packages/jax/_src/dispatch.py:95
DEBUG:jax._src.interpreters.pxla:Compiling convert_element_type with global shapes and types [ShapedArray(int32[])]. Argument mapping: (UnspecifiedValue,).
DEBUG:jax._src.dispatch:Finished jaxpr to MLIR module conversion jit(convert_element_type) in 0.031832933 sec
DEBUG:jax._src.compiler:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.compiler:get_compile_options XLA-AutoFDO profile: using XLA-AutoFDO profile version -1
DEBUG:jax._src.cache_key:get_cache_key hash of serialized computation: 4dc58c9b9b531cb92397bd6fc39852b8acae9947ab09fea8f0726ce246352480
DEBUG:jax._src.cache_key:get_cache_key hash after serializing computation: 4dc58c9b9b531cb92397bd6fc39852b8acae9947ab09fea8f0726ce2463

In [ ]:
%%time
# first time slow (due to compile)
_ = test_fun(a)

DEBUG:jax._src.dispatch:Finished tracing + transforming real for pjit in 0.000720024 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming _reduce_max for pjit in 0.001995087 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming isfinite for pjit in 0.001385689 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming _reduce_sum for pjit in 0.001323462 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming absolute for pjit in 0.000793457 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming abs for pjit in 0.008253098 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming subtract for pjit in 0.003425121 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming _reduce_max for pjit in 0.003085613 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming _reduce_sum for pjit in 0.002260447 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming test_fun for pjit in 0.340641260 sec
  never seen function:
    test_fun id=133816218246384 defined at <

CPU times: user 1.03 s, sys: 119 ms, total: 1.15 s
Wall time: 1.62 s


In [ ]:
import jax
import jax.numpy as jnp
from jax import jit
import numpy as np
import timeit
from jax import export
import pickle
from pathlib import Path


def complex_kernel(arr):
    multiplier = 2.5
    # Element-wise multiplication
    arr = arr * multiplier

    # Apply sine transformation
    arr = jnp.sin(arr)

    # Normalize the array to mean 0 and standard deviation 1
    mean = jnp.mean(arr)
    std_dev = jnp.std(arr)

    for _ in range(100):
      _ = (arr - mean) / std_dev

    arr = (arr - mean) / std_dev

    return arr

# Test the complex kernel on GPU with varying input lengths
def test_complex_kernel():
    # Different lengths to test
    array_lengths = [10, 100, 1000, 10000]
    evaluations = 10  # Number of evaluations for each array length

    for length in array_lengths:
        # Generate a random 1D array of the given length
        arr = jnp.array(np.random.randn(length), dtype=jnp.float32)

        out_file_name = f'test_array_size_{length}.pkl'

        if Path(out_file_name).exists():
          with open(out_file_name, 'rb') as f:
            serialized = pickle.load(f)
        else:
          exported = export.export(jax.jit(complex_kernel))(
              jax.ShapeDtypeStruct((), jax.ShapedArray(np.float32, shape=length)))
          serialized = exported.serialize()

          if not Path(out_file_name).exists():
            with open(out_file_name, 'wb') as f:
                pickle.dump(serialized, f)

        rehydrated_exp = export.deserialize(serialized)

        # Define a wrapper for the timeit function
        def kernel_evaluation():
            # return complex_kernel(arr).block_until_ready()  # Ensure GPU sync for timing
            return rehydrated_exp.call(arr).block_until_ready()  # Ensure GPU sync for timing

        first_iteration_time = timeit.timeit(kernel_evaluation, number=1)

        remaining_time = timeit.timeit(kernel_evaluation, number=evaluations - 1)
        avg_remaining_time = remaining_time / (evaluations - 1)
        total_time = first_iteration_time + remaining_time

        # Print timing results
        print(f"Array Length: {length}")
        print(f"Total time for {evaluations} evaluations: {total_time:f} seconds")
        print(f"Time for first compilation: {first_iteration_time}")
        print(f"Average time for remaining evaluation: {avg_remaining_time:f} seconds\n")

# Run the test case
test_complex_kernel()


ValueError: Rank mismatch for args[0]: expected () and called with (10,)

In [ ]:
from jax._src.lib import xla_client
getattr(xla_client.Client, "supports_executable_serialization", True)

True